In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.impute import KNNImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import numpy as np

# Load the data
data = pd.read_csv('TSS_data.csv')

# Impute NaN values for the entire dataset using KNN
imputer = KNNImputer(n_neighbors=5)
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Separate features and target variables
X = data_imputed.iloc[:, :-3]  # Exclude the last three columns (additional targets)
y_tss = data_imputed['TSS']
y_chla = data_imputed['Chla']
y_cdom = data_imputed['CDOM']

# Standardize the features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

def create_mdn_model(units=64, num_components=3, learning_rate=0.001):
    model = Sequential([
        Dense(units, activation='relu', input_dim=X_scaled.shape[1]),
        Dense(num_components * 3, activation='linear')  # 3 parameters per component
    ])

    def mdn_loss(y_true, y_pred):
        # Extract mean, log_sigma, and alpha using indexing
        num_params = num_components * 3
        mean = y_pred[:, :num_components]
        log_sigma = y_pred[:, num_components:2*num_components]
        alpha = y_pred[:, 2*num_components:]

        sigma = K.exp(log_sigma)

        # Gaussian probability density function
        pdf = K.exp(-0.5 * K.square((y_true - mean) / (sigma + 1e-8))) / (sigma + 1e-8) / np.sqrt(2 * np.pi)

        # Mixture of Gaussians
        loss = -K.log(K.sum(alpha * pdf, axis=1, keepdims=True) + 1e-8)
        return loss

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=mdn_loss)
    return model

# Perform 10-fold cross-validation with different hyperparameter combinations for TSS
kf = KFold(n_splits=10, shuffle=True, random_state=42)

param_combinations = [(32, 2, 0.001), (64, 2, 0.001), (128, 2, 0.001),
                      (32, 3, 0.001), (64, 3, 0.001), (128, 3, 0.001),
                      (32, 4, 0.001), (64, 4, 0.001), (128, 4, 0.001)]

def cross_validate_and_save(X, y, target_name, param_combinations):
    mape_scores = []

    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]

        for units, num_components, learning_rate in param_combinations:
            mdn_model = create_mdn_model(units=units, num_components=num_components, learning_rate=learning_rate)

            # Implement early stopping
            early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

            mdn_model.fit(
                X_train, y_train,
                epochs=100,
                batch_size=32,
                validation_data=(X_test, y_test),
                callbacks=[early_stopping],
                verbose=0
            )

            y_pred = mdn_model.predict(X_test)

            mape = mean_absolute_percentage_error(y_test, y_pred[:, :1].flatten())  # Use mean for prediction
            mape_scores.append((units, num_components, learning_rate, mape))

            # Save predicted values
            predictions_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred[:, :1].flatten()})
            predictions_df.to_csv(f'{target_name}_predictions_fold_{fold}.csv', index=False)

    # Find the best hyperparameters based on the lowest average MAPE
    best_hyperparameters = min(mape_scores, key=lambda x: x[3])
    best_units, best_num_components, best_learning_rate, best_mape = best_hyperparameters

    # Print the best hyperparameters and average MAPE
    print(f'Best Hyperparameters for {target_name}: Units={best_units}, Num Components={best_num_components}, Learning Rate={best_learning_rate}')
    print(f'Average MAPE for {target_name} across 10 folds: {best_mape}')

    return best_hyperparameters, best_mape

# Cross-validate and save models for each target variable
best_params_tss, mape_tss = cross_validate_and_save(X_scaled, y_tss, 'TSS', param_combinations)
best_params_chla, mape_chla = cross_validate_and_save(X_scaled, y_chla, 'Chla', param_combinations)
best_params_cdom, mape_cdom = cross_validate_and_save(X_scaled, y_cdom, 'CDOM', param_combinations)